## Import libraries and get data as letters:

In [18]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score

test = True

x_train = pd.read_csv('train.csv')

x_train = x_train[0:50000]

letters = x_train['Sequence'].apply(lambda x: pd.Series(list(x)))
x_train = x_train.drop('Sequence', axis = 1)
x_train = x_train.join(letters)
print(x_train)

if test:
    x_test = pd.read_csv('test.csv')
    letters = x_test['Sequence'].apply(lambda x: pd.Series(list(x)))
    x_test = x_test.drop('Sequence', axis = 1)
    x_test = x_test.join(letters)
    print(x_test)

       Active  0  1  2  3
0           0  D  K  W  L
1           0  F  C  H  N
2           0  K  D  Q  P
3           0  F  N  W  I
4           0  N  K  R  M
...       ... .. .. .. ..
49995       0  K  P  P  Q
49996       0  E  K  M  V
49997       0  A  K  S  L
49998       0  W  Y  D  P
49999       0  S  C  Q  P

[50000 rows x 5 columns]
       0  1  2  3
0      H  W  F  K
1      M  W  P  W
2      A  L  D  V
3      N  T  L  G
4      L  H  Y  Y
...   .. .. .. ..
47995  N  R  W  M
47996  M  M  M  K
47997  A  F  N  M
47998  C  R  Y  I
47999  M  K  F  C

[48000 rows x 4 columns]


## One hot encoding:

In [19]:
x_train_oh = pd.get_dummies(data=x_train, columns=[0, 1, 2, 3], sparse=True)
print(x_train_oh)

if test:
    x_test_oh = pd.get_dummies(data=x_test, columns=[0, 1, 2, 3], sparse=True)
    print(x_test_oh)

       Active  0_A  0_C  0_D  0_E  0_F  0_G  0_H  0_I  0_K  ...  3_M  3_N  \
0           0    0    0    1    0    0    0    0    0    0  ...    0    0   
1           0    0    0    0    0    1    0    0    0    0  ...    0    1   
2           0    0    0    0    0    0    0    0    0    1  ...    0    0   
3           0    0    0    0    0    1    0    0    0    0  ...    0    0   
4           0    0    0    0    0    0    0    0    0    0  ...    1    0   
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
49995       0    0    0    0    0    0    0    0    0    1  ...    0    0   
49996       0    0    0    0    1    0    0    0    0    0  ...    0    0   
49997       0    1    0    0    0    0    0    0    0    0  ...    0    0   
49998       0    0    0    0    0    0    0    0    0    0  ...    0    0   
49999       0    0    0    0    0    0    0    0    0    0  ...    0    0   

       3_P  3_Q  3_R  3_S  3_T  3_V  3_W  3_Y  
0        0    0    0    0  

In [4]:
# TODO: fix unbalanced data for better performance. Easiest is to oversample the 1's. Its worth a shot at least

In [5]:
# Implement Neural network <-- there is documentation in scikit
l1_size = int(0.006 * x_train_oh.shape[0])
l2_size = int(0.008 * x_train_oh.shape[0])
clf = MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(l1_size, l2_size), random_state=1, activation='tanh', max_iter=200)

### CV F1 score:

In [5]:
kf = KFold(n_splits=5)
f1_scores = []
for train_indices, test_indices in kf.split(x_train):
    clf.fit(x_train_oh.iloc[train_indices, 1:], x_train_oh.iloc[train_indices, 0])
    pred = clf.predict(x_train_oh.iloc[test_indices, 1:])
    #print(clf.score(x_train_oh.iloc[test_indices, 1:], x_train_oh.iloc[test_indices, 0]))
    f1_scores.append(f1_score(pred, x_train_oh.iloc[test_indices, 0]))
    print(f1_score(pred, x_train_oh.iloc[test_indices, 0]))
print("Mean F1 score:", np.array(f1_scores).mean())

0.8964241676942046


/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.7761194029850748


/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-6a2fb071fe73>", line 5, in <module>
    pred = clf.predict(x_train_oh.iloc[test_indices, 1:])
  File "/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 971, in predict
    y_pred = self._predict(X)
  File "/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 685, in _predict
    self._forward_pass(activations)
  File "/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 109, in _forward_pass
    activations[i + 1] = hidden_activation(activations[i + 1])
  File "/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_base.py"

KeyboardInterrupt: 

In [ ]:
# TODO: plot convergence of solver

### Predict on test data:

In [6]:
clf.fit(x_train_oh.iloc[:, 1:], x_train_oh['Active'])
pred = clf.predict(x_test_oh)
print(pred)

[0 0 0 ... 0 0 0]


In [17]:
df = pd.DataFrame(pred) 
print(df)
df.to_csv('out.csv', index=False, header=False)

       0
0      0
1      0
2      0
3      0
4      0
...   ..
47995  0
47996  0
47997  0
47998  0
47999  0

[48000 rows x 1 columns]


In [14]:
print(df[0:10])
print(pred[0:10])

   0
0  0
1  0
2  0
3  0
4  0
5  1
6  0
7  0
8  0
9  0
[0 0 0 0 0 1 0 0 0 0]


### Search to find parameters that give best F1 score:

In [ ]:
l1_size = int(0.006 * x_train_oh.shape[0])
l2_size = int(0.008 * x_train_oh.shape[0])

mlp = MLPClassifier(random_state=1, alpha=1e-3, hidden_layer_sizes=(l1_size, l2_size), max_iter=100, solver='adam', activation='tanh')

# All parameters we want to try:
parameter_space = {
    #'hidden_layer_sizes': [(60, 80), (50, 80), (70, 80), (60, 70), (60, 90)],
    #'activation': ['tanh', 'relu', 'logistic'],
    #'solver': ['sgd', 'adam'],
    'alpha': [1e-3, 1e-2, 1e-4],
}

# 10000 * 0.006

# Do grid search over all parameter options:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5, scoring='f1')
clf.fit(x_train_oh.iloc[:, 1:], x_train_oh['Active'])
print('Parameters', clf.best_params_, "gives best score:", clf.best_score_)
print(clf.cv_results_)

In [26]:
#  "the optimal size of the hidden layer is usually between the size of the input and size of the output layers"
#  "number of neurons = 0.005 * number of samples?" 2/3 size of input is also usual

# Easy: 0.607427055703
# Medium: 0.852643419573
# Hard: 0.89591280654

{'mean_fit_time': array([509.57949848, 157.87978411, 489.60818777]), 'std_fit_time': array([ 2.98684811,  3.92541923, 63.42419792]), 'mean_score_time': array([0.48849678, 0.39324956, 0.36429005]), 'std_score_time': array([0.03496291, 0.0267419 , 0.1384439 ]), 'param_activation': masked_array(data=['tanh', 'relu', 'logistic'],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'activation': 'tanh'}, {'activation': 'relu'}, {'activation': 'logistic'}], 'split0_test_score': array([0.86178862, 0.80911681, 0.81564246]), 'split1_test_score': array([0.86178862, 0.81609195, 0.82085561]), 'split2_test_score': array([0.83661972, 0.85553997, 0.80058224]), 'split3_test_score': array([0.86072423, 0.84094053, 0.80504909]), 'split4_test_score': array([0.84475524, 0.81818182, 0.75375375]), 'mean_test_score': array([0.85313529, 0.82797422, 0.79917663]), 'std_test_score': array([0.01049134, 0.01743948, 0.02383611]), 'rank_test_score': array([1, 2, 3], 